I am taking a simple greedy approach here. I take each item in the increasing order of weight (smallest first) and try to place it in a bag. The logic here is that small items leave more space for other items to be placed. To select which bag to place it in, I use a naive (read lazy) approach here, I place it the heaviest bag which can fit this item (total weight of bag cant exceed 50). This is sort of greedy, it places the smallest item in the heaviest bag, which leaves more space for the rest of items.

In [ ]:
# Import Libraries
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

Read Data, split it into Gift Type and ID

In [ ]:
d=pd.read_csv("../input/gifts.csv")
d['type'] = d['GiftId'].apply(lambda x: x.split('_')[0])
d['id'] = d['GiftId'].apply(lambda x: x.split('_')[1])

Define a function to add weights according to the distributions given. I got this part from "https://www.kaggle.com/mchirico/santas-uncertain-bags/santa-quick-look"

In [ ]:
def Weight(mType):
    if mType == "horse":
        return max(0, np.random.normal(5,2,1)[0])
    if mType == "ball":
        return max(0, 1 + np.random.normal(1,0.3,1)[0])
    if mType == "bike":
        return max(0, np.random.normal(20,10,1)[0])
    if mType == "train":
        return max(0, np.random.normal(10,5,1)[0])
    if mType == "coal":
        return 47 * np.random.beta(0.5,0.5,1)[0]
    if mType == "book":
        return np.random.chisquare(2,1)[0]
    if mType == "doll":
        return np.random.gamma(5,1,1)[0]
    if mType == "blocks":
        return np.random.triangular(5,10,20,1)[0]
    if mType == "gloves":
        return 3.0 + np.random.rand(1)[0] if np.random.rand(1) < 0.3 else np.random.rand(1)[0]

Applying weights to every gift item

In [ ]:
d['weight'] = d['type'].apply(lambda x: Weight(x))

This is the important part. Sorting the gifts in increasing order of their weights

In [ ]:
sorted_weights = d.sort_values(by=['weight'])

Creating a dataframe for bags, each row represents a bag, and a column called "bag_weights" gives the total leftover weight that can be filled up in each bag. Column "gifts" is where we store the list of gifts put in each bag. When initializing this dataframe, it will have all bag_weights as 50 and gifts as empty lists (all bags are empty).

In [ ]:
data = pd.DataFrame(pd.np.empty((1000, 1)) * pd.np.nan, columns = ['bag_weights']) 
data['bag_weights'] = 50.0000
data['gifts'] = np.empty((1000, 0)).tolist()

Iterating through each gift, checking and adding it to the heaviest bag it fits in. 

In [ ]:
for idx, row in sorted_weights.iterrows():
	data = data.sort_values(by=['bag_weights'], ascending = 0)
	for bag_idx, bag in data.iterrows():
		if row.weight <= bag.bag_weights:
			bag.gifts.append(row.GiftId)
			data.set_value(bag_idx,'bag_weights',float(bag.bag_weights) - float(row.weight))
			break

Putting output in a list

In [ ]:
packed_bags = [[]]
for bag_idx, bag in data.iterrows():
	if len(bag.gifts)>=3:
		packed_bags.append(" ".join(bag.gifts))